Ref Articles 
- https://towardsdatascience.com/custom-transformers-and-ml-data-pipelines-with-python-20ea2a7adb65
- https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/
- https://discuss.analyticsvidhya.com/t/what-is-the-difference-between-predict-and-predict-proba/67376
- https://github.com/AnilBetta/AV-Janata-Hack-healh-Care-2/blob/master/av-jh-hca2-cat.ipynb
- https://github.com/gcspkmdr/HA-Hackathon

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
#from catboost import CatBoostClassifier
#from sklearn.model_selection import StratifiedKFold,KFold,GroupKFold
#from sklearn.metrics import accuracy_score

#Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector

#For Missing Value and Feature Engineering
from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder, MinMaxScaler

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

import time


## Import Data

In [3]:
train = pd.read_csv("training.csv")
test = pd.read_csv("test.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   RefId                              72983 non-null  int64  
 1   IsBadBuy                           72983 non-null  int64  
 2   PurchDate                          72983 non-null  object 
 3   Auction                            72983 non-null  object 
 4   VehYear                            72983 non-null  int64  
 5   VehicleAge                         72983 non-null  int64  
 6   Make                               72983 non-null  object 
 7   Model                              72983 non-null  object 
 8   Trim                               70623 non-null  object 
 9   SubModel                           72975 non-null  object 
 10  Color                              72975 non-null  object 
 11  Transmission                       72974 non-null  obj

## SMOTE

In [5]:
#insert code

## Feat Engineering

In [6]:
# Date
#PurchDate

In [7]:
train['mean_MMRAcquisitionAuctionAveragePrice_Make']=train.groupby(['Make'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')
train['mean_MMRAcquisitionAuctionAveragePrice_Model']=train.groupby(['Model'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')
train['mean_MMRAcquisitionAuctionAveragePrice_Trim']=train.groupby(['Trim'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')
train['mean_MMRAcquisitionAuctionAveragePrice_SubModel']=train.groupby(['SubModel'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')
train['mean_MMRAcquisitionAuctionAveragePrice_Color']=train.groupby(['Color'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')
train['mean_MMRAcquisitionAuctionAveragePrice_Transmission']=train.groupby(['Transmission'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')

train['mean_MMRAcquisitionAuctionCleanPrice_Make']=train.groupby(['Make'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')
train['mean_MMRAcquisitionAuctionCleanPrice_Model']=train.groupby(['Model'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')
train['mean_MMRAcquisitionAuctionCleanPrice_Trim']=train.groupby(['Trim'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')
train['mean_MMRAcquisitionAuctionCleanPrice_SubModel']=train.groupby(['SubModel'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')
train['mean_MMRAcquisitionAuctionCleanPrice_Color']=train.groupby(['Color'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')
train['mean_MMRAcquisitionAuctionCleanPrice_Transmission']=train.groupby(['Transmission'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')

In [8]:
test['mean_MMRAcquisitionAuctionAveragePrice_Make']=test.groupby(['Make'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')
test['mean_MMRAcquisitionAuctionAveragePrice_Model']=test.groupby(['Model'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')
test['mean_MMRAcquisitionAuctionAveragePrice_Trim']=test.groupby(['Trim'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')
test['mean_MMRAcquisitionAuctionAveragePrice_SubModel']=test.groupby(['SubModel'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')
test['mean_MMRAcquisitionAuctionAveragePrice_Color']=test.groupby(['Color'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')
test['mean_MMRAcquisitionAuctionAveragePrice_Transmission']=test.groupby(['Transmission'])['MMRAcquisitionAuctionAveragePrice'].transform('mean')

test['mean_MMRAcquisitionAuctionCleanPrice_Make']=test.groupby(['Make'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')
test['mean_MMRAcquisitionAuctionCleanPrice_Model']=test.groupby(['Model'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')
test['mean_MMRAcquisitionAuctionCleanPrice_Trim']=test.groupby(['Trim'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')
test['mean_MMRAcquisitionAuctionCleanPrice_SubModel']=test.groupby(['SubModel'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')
test['mean_MMRAcquisitionAuctionCleanPrice_Color']=test.groupby(['Color'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')
test['mean_MMRAcquisitionAuctionCleanPrice_Transmission']=test.groupby(['Transmission'])['MMRAcquisitionAuctionCleanPrice'].transform('mean')

## Divide Dataset into X and Y

In [14]:
#create X and y datasets for splitting 
X = train.drop(['RefId', 'IsBadBuy'], axis=1)
y = train['IsBadBuy']

In [16]:
all_features = X.columns

In [17]:
all_features = all_features.tolist()

In [18]:
#import train_test_split library
from sklearn.model_selection import train_test_split

# create train test split
X_train, X_test, y_train, y_test = train_test_split( X,  y, test_size=0.3, random_state=0)  

## Setup Pipeline 

In [19]:
preprocessor = make_column_transformer(
    
    (make_pipeline(
    SimpleImputer(strategy = 'median'),
    MinMaxScaler()), make_column_selector(dtype_include=np.number)),
    
    (make_pipeline(
    SimpleImputer(strategy = 'constant', fill_value = 'missing'),
    OneHotEncoder(categories = 'auto', handle_unknown = 'ignore')), make_column_selector(dtype_include=object)),
    
)

In [21]:
preprocessor_best = make_pipeline(preprocessor, 
                                  #SelectKBest(chi2, k = 50)
                                 VarianceThreshold())

In [22]:
RF_Model = make_pipeline(preprocessor_best, RandomForestClassifier(n_estimators = 100))

## Grid Search

In [23]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 50)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,4,6,8]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 8]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [24]:
# Create the param grid
param_grid = {'randomforestclassifier__n_estimators': n_estimators,
               'randomforestclassifier__max_features': max_features,
               'randomforestclassifier__max_depth': max_depth,
               'randomforestclassifier__min_samples_split': min_samples_split,
               'randomforestclassifier__min_samples_leaf': min_samples_leaf,
               'randomforestclassifier__bootstrap': bootstrap
             }
print(param_grid)

{'randomforestclassifier__n_estimators': [100, 118, 136, 155, 173, 191, 210, 228, 246, 265, 283, 302, 320, 338, 357, 375, 393, 412, 430, 448, 467, 485, 504, 522, 540, 559, 577, 595, 614, 632, 651, 669, 687, 706, 724, 742, 761, 779, 797, 816, 834, 853, 871, 889, 908, 926, 944, 963, 981, 1000], 'randomforestclassifier__max_features': ['auto', 'sqrt'], 'randomforestclassifier__max_depth': [2, 4, 6, 8], 'randomforestclassifier__min_samples_split': [2, 5, 8], 'randomforestclassifier__min_samples_leaf': [1, 2, 3], 'randomforestclassifier__bootstrap': [True, False]}


In [25]:
from sklearn.model_selection import RandomizedSearchCV
rf_RandomGrid = RandomizedSearchCV(estimator = RF_Model, param_distributions = param_grid, cv = 10, verbose=1, n_jobs = -1)

In [26]:
rf_RandomGrid.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  7.4min finished


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('pipeline',
                                              Pipeline(steps=[('columntransformer',
                                                               ColumnTransformer(transformers=[('pipeline-1',
                                                                                                Pipeline(steps=[('simpleimputer',
                                                                                                                 SimpleImputer(strategy='median')),
                                                                                                                ('minmaxscaler',
                                                                                                                 MinMaxScaler())]),
                                                                                                <sklearn.compose._column_transformer.make_column_selector object at 0x0000018853FED3C8>

## Accuracy

In [27]:
print(f'Train : {rf_RandomGrid.score(X_train, y_train):.3f}')
print(f'Test : {rf_RandomGrid.score(X_test, y_test):.3f}')

Train : 0.897
Test : 0.899


## Gini Index

In [28]:
def gini(actual, pred):
    assert (len(actual) == len(pred))
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    totalLosses = all[:, 0].sum()
    giniSum = all[:, 0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)


def gini_normalized(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

In [29]:
actual_train = y_train
pred_train = rf_RandomGrid.predict(X_train)
actual_test = y_test
pred_test = rf_RandomGrid.predict(X_test)

In [30]:
print(f'Gini Train : {gini(actual_train,pred_train):.3f}')
print(f'Gini Test : {gini(actual_test,pred_test):.3f}')

Gini Train : 0.098
Gini Test : 0.105


## Submission 

In [31]:
test_pred = rf_RandomGrid.predict_proba(test[X.columns])[:,1]

In [32]:
AllSub = pd.DataFrame({ 'RefId': test['RefId'],
                       'IsBadBuy' : test_pred
    
})

In [33]:
AllSub['IsBadBuy'] = AllSub['IsBadBuy'].apply(lambda x: 1 if x > 0.09 else 0)

In [34]:
AllSub.to_csv('PyCaret_RF_Pipe_Better_Pipe_more.csv', index = False)

In [ ]:
# 0.12957 